<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

# **COVID Dashboard**

## 1\. Contexto

Escreva uma breve descrição do problema.

## 2\. Pacotes e bibliotecas

In [ ]:
# importe todas as suas bibliotecas aqui, siga os padrões do PEP8:
#
# - 1º pacotes nativos do python: json, os, etc.;
# - 2º pacotes de terceiros: pandas, seabornm etc.;
# - 3º pacotes que você desenvolveu.
#

import math
from typing import Iterator
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

## 3\. Extração

### Dataset de Casos de Covid

In [ ]:
# faça o código de extração dos dados:
#
# - coleta de dados;
# - wrangling da estrutura;
# - exploração do schema;
# - etc.
...

Ellipsis

In [ ]:
# Fazendo o consumo do Dataset de casos de Covid 19.
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_daily_reports/01-01-2021.csv', sep=',')

In [ ]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-02 05:22:33,33.93911,67.709953,52513,2201,41727,8585,Afghanistan,134.896578,4.191343
1,NaN,NaN,NaN,Albania,2021-01-02 05:22:33,41.15330,20.168300,58316,1181,33634,23501,Albania,2026.409062,2.025173
2,NaN,NaN,NaN,Algeria,2021-01-02 05:22:33,28.03390,1.659600,99897,2762,67395,29740,Algeria,227.809861,2.764848
3,NaN,NaN,NaN,Andorra,2021-01-02 05:22:33,42.50630,1.521800,8117,84,7463,570,Andorra,10505.403482,1.034865
4,NaN,NaN,NaN,Angola,2021-01-02 05:22:33,-11.20270,17.873900,17568,405,11146,6017,Angola,53.452981,2.305328


In [ ]:
# Informações do Dataframe
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4011 entries, 0 to 4010
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FIPS                 3265 non-null   float64
 1   Admin2               3270 non-null   object 
 2   Province_State       3833 non-null   object 
 3   Country_Region       4011 non-null   object 
 4   Last_Update          4011 non-null   object 
 5   Lat                  3922 non-null   float64
 6   Long_                3922 non-null   float64
 7   Confirmed            4011 non-null   int64  
 8   Deaths               4011 non-null   int64  
 9   Recovered            4011 non-null   int64  
 10  Active               4011 non-null   int64  
 11  Combined_Key         4011 non-null   object 
 12  Incident_Rate        3922 non-null   float64
 13  Case_Fatality_Ratio  3963 non-null   float64
dtypes: float64(5), int64(4), object(5)
memory usage: 438.8+ KB


In [ ]:
# Função para gerar uma sequência de datas a partir de uma data de início até uma data de fim
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [ ]:
# Definindo as datas dos dados.
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [ ]:
# Criando um laço para extrair as datas do repositório GIT, de acordo com a data inicial e a data final.
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], axis=0, ignore_index=True)

In [ ]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01



### Dataset de Vacinação do Covid 19.

In [ ]:
# Fazendo o consumo do Dataset de vacinação de Covid 19.
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3])

In [ ]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


In [ ]:
# Selecionando somente os dados do Brasil e as colunas a serem utilizadas para as análises.
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [ ]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1674 non-null   object        
 1   population               1674 non-null   int64         
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1674 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 91.7+ KB


## 4\. Transformação

In [ ]:
# faça o código de manipulação dos dados:
#
# - enriquecimento;
# - controle de qualidade;
# - etc.

...

Ellipsis

### Data Wrangling - Dataframe 'cases'

In [ ]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [ ]:
# Mudando nomes de algumas colunas e deixando todas as colunas em minúsculo.
cases = cases.rename(
    columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [ ]:
# Criando um dicionário para alterar algumas nomes de estados
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

# Criando uma condição para mudar linha por linha
cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [ ]:
# Criando mais duas colunas para enriquecer os dados
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year'] = cases['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
# Criando a coluna de população
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [ ]:
cases.head()

,state,country,confirmed,deaths,date,month,year,population
0,Acre,Brazil,41689,796,2021-01-01,2021-01,2021,881935.0
1,Alagoas,Brazil,105091,2496,2021-01-01,2021-01,2021,3337357.0
2,Amapá,Brazil,68361,926,2021-01-01,2021-01,2021,845731.0
3,Amazonas,Brazil,201574,5295,2021-01-01,2021-01,2021,4144597.0
4,Bahia,Brazil,494684,9159,2021-01-01,2021-01,2021,14873064.0


In [ ]:
# Este bloco de código analisa a tendência de casos confirmados e mortes por COVID-19 em cada estado.
# Para cada estado único na coluna 'state' do DataFrame 'cases', ele executa os seguintes passos:
# 1. Extrai os dados apenas daquele estado e ordena por data para manter a sequência temporal.
# 2. Calcula a variação diária de casos confirmados ('confirmed_1d') e a média móvel de 7 dias
#    para suavizar as variações de curto prazo ('confirmed_moving_avg_7d').
# 3. Calcula a taxa entre a média móvel atual de 7 dias e a média móvel de 7 dias de 14 dias atrás
#    ('confirmed_moving_avg_7d_rate_14d') para identificar mudanças de longo prazo nos casos.
# 4. Com base nessa taxa, classifica a tendência de casos em 'downward' (queda), 'stable' (estável),
#    ou 'upward' (aumento) usando a função 'get_trend'.
# 5. Repete os mesmos cálculos para a coluna de mortes, calculando variação diária de mortes
#    ('deaths_1d'), média móvel de 7 dias ('deaths_moving_avg_7d') e a taxa de 14 dias
#    ('deaths_moving_avg_7d_rate_14d') para determinar a tendência de mortes ('deaths_trend').
# 6. Concatena os resultados para cada estado em um único DataFrame, 'cases_', que armazena todas
#    as tendências de casos e mortes. Ao final, a variável 'cases' é atualizada com esse DataFrame.

# Este código permite observar o comportamento temporal da pandemia em diferentes estados,
# destacando se a situação de casos e mortes está melhorando, piorando ou estável ao longo do tempo.


cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:    # Se a divisão for menor que 0.75 (correção 0.85)
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state],axis=0, ignore_index=True)

cases = cases_
cases_ = None

In [ ]:
cases.head()

,state,country,confirmed,deaths,date,month,year,population,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend
0,Acre,Brazil,41689,796,2021-01-01,2021-01,2021,881935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acre,Brazil,41941,798,2021-01-02,2021-01,2021,881935.0,252.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
2,Acre,Brazil,42046,802,2021-01-03,2021-01,2021,881935.0,105.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,Acre,Brazil,42117,806,2021-01-04,2021-01,2021,881935.0,71.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
4,Acre,Brazil,42170,808,2021-01-05,2021-01,2021,881935.0,53.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [ ]:
# Mudando os tipos das colunas.
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [ ]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   state                             9828 non-null   object        
 1   country                           9828 non-null   object        
 2   confirmed                         9828 non-null   int64         
 3   deaths                            9828 non-null   int64         
 4   date                              9828 non-null   datetime64[ns]
 5   month                             9828 non-null   object        
 6   year                              9828 non-null   object        
 7   population                        9828 non-null   Int64         
 8   confirmed_1d                      9801 non-null   Int64         
 9   confirmed_moving_avg_7d           9639 non-null   Int64         
 10  confirmed_moving_avg_7d_rate_14d  9261 non-null 

In [ ]:
# Selecionando somente as colunas que iremos fazer nossa análise.
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [ ]:
cases.head(25)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


### Data Wrangling - Dataframe 'vaccines'

In [ ]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


In [ ]:
# Preenchendo valores nulos, com valores conhecidos nas colunas, que não mudam abruptamente
vaccines = vaccines.ffill()

In [ ]:
# Selecionando uma data específica para análise
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

In [ ]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-01
1,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-02
2,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-03
3,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-04
4,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-05


In [ ]:
# Mudando nomes de colunas
vaccines = vaccines.rename(
    columns={
        'total_vaccinations': 'total',
        'people_vaccinated': 'one_shot',
        'people_fully_vaccinated': 'two_shots',
        'total_boosters': 'three_shots'
  }
)

In [ ]:
# Criando duas novas colunas de mês e ano
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year'] = vaccines['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
# Criando três colunas que vão nos fornecer de forma aproximada as pessoas que tomaram uma dose, duas doses e três doses
vaccines['one_shot'] = round(100000 * (vaccines['one_shot'] / vaccines['population']))
vaccines['two_shots'] = round(100000 * (vaccines['two_shots'] / vaccines['population']))
vaccines['three_shots'] = round(100000 * (vaccines['three_shots'] / vaccines['population']))

In [ ]:
# Mudando o tipo de cada coluna
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [ ]:
# Organizando o dataframe
vaccines = vaccines[['date', 'location', 'population', 'total', 'one_shot', 'two_shots', 'three_shots', 'month', 'year']]

In [ ]:
vaccines.tail()

,date,location,population,total,one_shot,two_shots,three_shots,month,year
360,2021-12-27,Brazil,215313504,329011365,77075,66305,11713,2021-12,2021
361,2021-12-28,Brazil,215313504,329861730,77126,66399,11963,2021-12,2021
362,2021-12-29,Brazil,215313504,330718457,77163,66546,12177,2021-12,2021
363,2021-12-30,Brazil,215313504,331164041,77183,66600,12311,2021-12,2021
364,2021-12-31,Brazil,215313504,331273910,77188,66617,12341,2021-12,2021


## 5\. Carregamento

In [ ]:
# faça o código para persistência dos dados:
#
# - salve-os no formato csv sem índice;
# - etc.

...

Ellipsis

### Salvando o Dataframe de Casos depois do Wrangling.

In [ ]:
# Salvando o Dataframe 'cases' em um arquivo csv.
cases.to_csv('./covid-cases.csv', sep=',', index=False)

### Salvando o Dataframe de Vacinas depois do Wrangling.

In [ ]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

## 6. Dashboard dos Dados de Vacinação e de Casos.


[Dashboard Covid - Looker](https://lookerstudio.google.com/reporting/0ded50e2-3e3f-4488-9610-6b21e528d32d)